In [ ]:
# coding: utf-8
import torch
import torch.nn as nn
import random

from tqdm import tqdm
import time
# GPU利用
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


In [ ]:
####データローダー

In [ ]:
from ml.price.lstm import LSTMModel

# モデルのパラメータを設定
input_size = 1  # 時系列データの特徴量の数（この場合は1）
hidden_size = 50  # LSTMの隠れ状態のサイズ
output_size = 128  # 出力する潜在変数の次元数

# モデルのインスタンス化
model = LSTMModel(input_size, hidden_size, output_size)


In [4]:
from ml.price.transfomer import TransformerModelWithPositionalEncoding

# モデルのパラメータを設定
input_size = 1
hidden_size = 512
output_size = 128
nhead = 8
num_encoder_layers = 3
dim_feedforward = 2048

# モデルのインスタンス化
transformer_model_with_attention = TransformerModelWithPositionalEncoding(input_size, hidden_size, output_size, nhead, num_encoder_layers, dim_feedforward)


/home/macaroni/.conda/envs/torch201_py39/lib/python3.9/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [1]:
# coding: utf-8
import torch
import torch.nn as nn
import random

from tqdm import tqdm
import time
# GPU利用
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)



from ml.text.bert import BERT_Report

# 事前学習済みモデルのロード
from transformers import BertModel
bert = BertModel.from_pretrained('bert-base-uncased')
# tokenizer インスタンスの生成
# 対象モデルは'bert-base-uncased'
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# モデルインスタンスの生成
# 出力は感情分析なので2
OUTPUT_DIM = 64

model = BERT_Report(bert, OUTPUT_DIM, tokenizer).to(device)
model.eval()

text_2023_01_us = """
The US stock market sentiment in the first week of January 2023 showed a cautiously optimistic outlook, influenced by various factors including sector performances, rate hike expectations, and company-specific news.
1. **January Indicator Trifecta and Sector Performance**: The stock market experienced what is known as the "January Indicator Trifecta." This refers to a Santa Claus rally, positive first five days of January, and a positive January Barometer. The occurrence of all three indicators historically suggests a favorable market in the following 11 months. In terms of sector performance, Consumer Discretionary and Communication Services led the gains. The Nasdaq Composite showed a strong performance, especially in technology stocks, while small-cap stocks indicated by the S&P 600 Small Cap index also rose significantly.
2. **Federal Reserve and Rate Hike Odds**: The market was anticipating a 25 basis point rate increase at the February Federal Reserve meeting. This expectation was reflected in the pricing of fed fund futures. Treasury yields saw some weakness, with the 10-year Treasury yield dropping to 3.51%, which was below the October peak of 4.25%.
3. **Corporate Earnings and Stock Performance**: About one-third of S&P 500 companies reported a 5% decline in Q4 profits, compared to an expected 3.2% decline. Despite this, there were sectors like Energy, Industrials, and Consumer Discretionary that saw significant earnings growth. Notably, the worst-performing stocks of 2022 saw an average increase of 20.1% in early 2023, suggesting a short-term reversion of oversold stocks rather than a fundamental shift in market leadership.
4. **Influence of Company-Specific News**: Individual companies also influenced market sentiment. For instance, Tesla's shares went up after announcing price cuts in China, while Bed Bath & Beyond's shares declined significantly due to bankruptcy considerations. Costco's stock gained after reporting positive December sales data.
5. **Overall Market Dynamics**: The first week of January 2023 closed higher for US stocks, spurred by a favorable jobs report and corporate news. The CBOE Volatility Index (VIX), often regarded as a fear gauge, decreased by 11% in January, indicating a decrease in market volatility.
In summary, the first week of January 2023 in the US stock market was marked by a mix of optimism driven by sector performances and cautious sentiment due to economic indicators and corporate earnings. While there was a positive outlook based on the January indicators, the market remained sensitive to rate hikes and individual corporate performances.
References: 
- StockCharts.com【6†source】
- Nasdaq【7†source】
- Yahoo Finance【8†source】【9†source】
"""

encoding_text = model.encoding(text_2023_01_us)
input = encoding_text.input_ids.to(device)
predictions = model(input)
print(predictions.shape)

cuda
L28 torch.Size([768])
torch.Size([64])


In [7]:

from ml.text.text_concatenate import TEXT_Cconcatenate


learning_epoch_num = 10
# 合計値(total)を設定

all_reports = torch.cat( [predictions,predictions,predictions,predictions,predictions]  , dim=0)
print(all_reports.shape)  # 潜在ベクトルの形状を確認

input_size = 320
output_size = 128
text_cconcatenate = TEXT_Cconcatenate(input_size, output_size).to(device)
all_reports_latent = text_cconcatenate(all_reports)
print(all_reports_latent.shape)  # 潜在ベクトルの形状を確認



for i in tqdm(range(learning_epoch_num)):
    # ダミーの時系列データを生成（長さは15から20のランダム）
    sequence_length = random.randint(15, 20)
    dummy_time_series = torch.randn(sequence_length, 1, input_size)

    # Transformerモデルに入力
    # バッチサイズが1のため、[シーケンス長, バッチサイズ, 特徴量数]に変形
    dummy_time_series = dummy_time_series.permute(1, 0, 2)
    latent_vector = transformer_model_with_attention(dummy_time_series)
    print(latent_vector[0].shape)  # 潜在ベクトルの形状を確認


torch.Size([320])
torch.Size([128])


  0%|          | 0/10 [00:00<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (17x320 and 1x512)